In [1]:
from pipeline import *
from moviepy.editor import VideoFileClip
from IPython.display import HTML
from collections import Counter
from statistics import mean

In [2]:
orient = 11
pix_per_cell = 16
cell_per_block = 2
cspace = 'YUV'
sig = "_ori"+ str(orient)+ "_pix" + str(pix_per_cell) + "_cell_per_block"
clf = None
scaler = None

In [3]:
X, y = loadDataFull()
clf, scaler = trainModelFull(X, y)

Loading the features took 0.24 seconds


In [4]:
def pipeline(image):
    img = (image/255.0).astype("float32")
    heat = np.zeros_like(img[:,:,0]).astype(np.int)

    windows = slide_window(img, x_start_stop=[650, img.shape[1]+40],
        y_start_stop=[380, 650], xy_overlap=(0.65, 0.65))
    windows2 = slide_window(img, x_start_stop=[650, img.shape[1]+40],
        y_start_stop=[380, 650], xy_overlap=(0.75, 0.75), xy_window=(80, 80))
    windows3 = slide_window(img, x_start_stop=[650, img.shape[1]+40],
        y_start_stop=[380, 650], xy_overlap=(0.75, 0.75), xy_window=(96, 96))
    
    on_windows = search_windows(img, windows, clf, scaler,
                    orient=orient, pix_per_cell=pix_per_cell,
                    cell_per_block=cell_per_block, cspace=cspace)

    on_windows2 = search_windows(img, windows2, clf, scaler,
                    orient=orient, pix_per_cell=pix_per_cell,
                    cell_per_block=cell_per_block, cspace=cspace)

    on_windows3 = search_windows(img, windows3, clf, scaler,
                    orient=orient, pix_per_cell=pix_per_cell,
                    cell_per_block=cell_per_block, cspace=cspace)
    
    heatmap = add_heat(heat, on_windows)
    heatmap = add_heat(heatmap, on_windows2)
    heatmap = add_heat(heatmap, on_windows3)
    boxer.updateHeat(heatmap)
    boxer.num = boxer.num + 1
    if(boxer.num % 10 == 0):
        boxer.getAvg()
    if boxer.labels == None:
        draw_img = image
    else:
        draw_img = draw_labeled_bboxes(np.copy(image), boxer.labels)

    return draw_img

In [5]:
def save_pipeline(image):
    img = (image/255.0).astype("float32")
    if(boxer.num % 10 == 0 or boxer.num == 1):
        windows = slide_window(img, x_start_stop=[650, img.shape[1]+40],
            y_start_stop=[380, 650], xy_overlap=(0.65, 0.65))
        windows2 = slide_window(img, x_start_stop=[650, img.shape[1]+40],
            y_start_stop=[380, 650], xy_overlap=(0.75, 0.75), xy_window=(80, 80))
        windows3 = slide_window(img, x_start_stop=[650, img.shape[1]+40],
            y_start_stop=[380, 650], xy_overlap=(0.75, 0.75), xy_window=(96, 96))

        on_windows = search_windows(img, windows, clf, scaler,
                        orient=orient, pix_per_cell=pix_per_cell,
                        cell_per_block=cell_per_block, cspace=cspace)

        on_windows2 = search_windows(img, windows2, clf, scaler,
                        orient=orient, pix_per_cell=pix_per_cell,
                        cell_per_block=cell_per_block, cspace=cspace)

        on_windows3 = search_windows(img, windows3, clf, scaler,
                        orient=orient, pix_per_cell=pix_per_cell,
                        cell_per_block=cell_per_block, cspace=cspace)
        save_on_windows(img, on_windows, 1, boxer.num)
        save_on_windows(img, on_windows2, 2, boxer.num)
        save_on_windows(img, on_windows3, 3, boxer.num)
    
    boxer.num = boxer.num + 1

    return image

In [6]:
img_set = plt.imread("test_images_png/test0.png")
class Boxes():
    def __init__(self):
        # was the line detected in the last iteration?
        self.current_heat = None
        self.labels = None
        self.heatmap_avg = np.zeros_like(img_set[:,:,0]).astype(np.int)  
        self.num = 1
    def updateHeat(self,new_heat):
        self.heatmap_avg = new_heat + self.heatmap_avg
        return
    def resetHeat(self):
        self.heatmap_avg = self.heatmap_avg * 0
        return
    def getAvg(self):
        self.current_heat = self.heatmap_avg / 10 
        self.current_heat = apply_threshold(self.current_heat, 3)
        self.labels = label(self.current_heat)
        self.resetHeat()
        return self.current_heat
        
boxer = Boxes()

In [7]:
proj_video_out = 'project_video_output.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(pipeline) #NOTE: this function expects color images!!
%time white_clip.write_videofile(proj_video_out, audio=False)

[MoviePy] >>>> Building video project_video_yuh.mp4
[MoviePy] Writing video project_video_yuh.mp4


100%|███████████████████████████████████▉| 1260/1261 [1:03:48<00:02,  2.86s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_yuh.mp4 

Wall time: 1h 3min 50s
